### Konstantina Andronikou

## Comparison with notebook with_conjuction_BERT

## This noteboook is implementing a sentence without a conjuction

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
# To remove all warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Someone killed Mary.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'killed',
   'description': '[ARG0: Someone] [V: killed] [ARG1: Mary] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
 'words': ['Someone', 'killed', 'Mary', '.']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone killed Mary."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'killed',
     'description': '[ARG0: Someone] [V: killed] [ARG1: Mary] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
   'words': ['Someone', 'killed', 'Mary', '.']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name'],meta['last_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('Someone killed {first_name} {last_name}.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'last_name': 'Wilson', 'first_name': 'Donald'}, {'last_name': 'Wilson', 'first_name': 'Bobby'}, {'last_name': 'Turner', 'first_name': 'Alfred'}, {'last_name': 'Rogers', 'first_name': 'Jane'}, {'last_name': 'Alexander', 'first_name': 'Joe'}, {'last_name': 'Foster', 'first_name': 'Harold'}, {'last_name': 'Hughes', 'first_name': 'Sharon'}, {'last_name': 'Cook', 'first_name': 'Adam'}, {'last_name': 'Wright', 'first_name': 'Pamela'}, {'last_name': 'Morris', 'first_name': 'Joseph'}, {'last_name': 'Miller', 'first_name': 'Alfred'}, {'last_name': 'Rose', 'first_name': 'Caroline'}, {'last_name': 'Stone', 'first_name': 'Scott'}, {'last_name': 'Ford', 'first_name': 'Jill'}, {'last_name': 'Ward', 'first_name': 'Evelyn'}, {'last_name': 'Williams', 'first_name': 'Virginia'}, {'last_name': 'Thomas', 'first_name': 'Julie'}, {'last_name': 'Nelson', 'first_name': 'Scott'}, {'last_name': 'Hart', 'first_name': 'Alex'}, {'last_name': 'Henderson', 'first_name': 'Lisa'}, {'last_name': 'Murray', 'first

In [12]:
with open('Dataset_BERT/Without_conjunction.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Without_con_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 999 examples
Test cases:      999
Fails (rate):    0 (0.0%)
['Someone', 'killed', 'Donald', 'Wilson', '.'] [ True]
['Someone', 'killed', 'Bobby', 'Wilson', '.'] [ True]
['Someone', 'killed', 'Alfred', 'Turner', '.'] [ True]
['Someone', 'killed', 'Jane', 'Rogers', '.'] [ True]
['Someone', 'killed', 'Joe', 'Alexander', '.'] [ True]
['Someone', 'killed', 'Harold', 'Foster', '.'] [ True]
['Someone', 'killed', 'Sharon', 'Hughes', '.'] [ True]
['Someone', 'killed', 'Adam', 'Cook', '.'] [ True]
['Someone', 'killed', 'Pamela', 'Wright', '.'] [ True]
['Someone', 'killed', 'Joseph', 'Morris', '.'] [ True]
['Someone', 'killed', 'Alfred', 'Miller', '.'] [ True]
['Someone', 'killed', 'Caroline', 'Rose', '.'] [ True]
['Someone', 'killed', 'Scott', 'Stone', '.'] [ True]
['Someone', 'killed', 'Jill', 'Ford', '.'] [ True]
['Someone', 'killed', 'Evelyn', 'Ward', '.'] [ True]
['Someone', 'killed', 'Virginia', 'Williams', '.'] [ True]
['Someone', 'killed', 'Julie', 'Thomas', '.'] [ True]
['Some

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_without_con_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 